In [2]:
import os

In [1]:
import warnings

import ray
import torch

warnings.filterwarnings("ignore")

In [ ]:
# Build a local ray cluster. The head node and worker node are on this machine
ray.init()

2025-06-29 10:07:52,758	INFO worker.py:1908 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.11.13', ray_version='2.47.1', ray_commit='61d3f2f1aa33563faa398105f4abda88cb39440b')

(GPUAccumulator pid=21097) rank 0, value: tensor([1.], device='cuda:0')
(GPUAccumulator pid=21218) rank 1, value: tensor([2.], device='cuda:0')
(GPUAccumulatorDecorator pid=21326) rank 0, value: tensor([10.], device='cuda:0')
(raylet) The autoscaler failed with the following error:
Terminated with signal 15
  File "/home/yuanz/miniconda3/envs/verl0624/lib/python3.11/site-packages/ray/autoscaler/_private/monitor.py", line 747, in <module>
    monitor.run()
  File "/home/yuanz/miniconda3/envs/verl0624/lib/python3.11/site-packages/ray/autoscaler/_private/monitor.py", line 604, in run
    self._run()
  File "/home/yuanz/miniconda3/envs/verl0624/lib/python3.11/site-packages/ray/autoscaler/_private/monitor.py", line 459, in _run
    time.sleep(AUTOSCALER_UPDATE_INTERVAL_S)

(GPUAccumulatorDecorator pid=21445) rank 1, value: tensor([11.], device='cuda:0')


## step 1 单进程 基于ray的处理方式

In [4]:
class Accumulator:
    def __init__(self):
        self.value = 0

    def add(self, x):
        self.value += x

    def get_value(self):
        return self.value


accumulator = Accumulator()
a1 = accumulator.get_value()
accumulator.add(10)
a2 = accumulator.get_value()


a1, a2


(0, 10)

In [5]:
@ray.remote
class Accumulator:
    def __init__(self):
        self.value = 0

    def add(self, x):
        self.value += x

    def get_value(self):
        return self.value


accumulator = Accumulator.remote()
type(accumulator)

ray.actor.ActorHandle

In [6]:
a1_ref = accumulator.get_value.remote()

accumulator.add.remote(10)
a2_ref = accumulator.get_value.remote()

ray.get(a1_ref), ray.get(a2_ref)

(0, 10)

## step 2 多线程的 基于verl封装的调用

In [8]:
from verl.single_controller.base import Worker
from verl.single_controller.ray.base import (
    RayClassWithInitArgs,
    RayResourcePool,
    RayWorkerGroup,
    merge_resource_pool,
)


In [9]:
resource_pool = RayResourcePool([2], use_gpu=True)
resource_pool


In [11]:
@ray.remote
class GPUAccumulator(Worker):
    def __init__(self):
        super().__init__()
        self.value = torch.zeros(1, device="cuda") + self.rank


    def add(self, x):
        self.value += x
        print(f"rank {self.rank}, value: {self.value}")
        return self.value.cpu()

In [12]:
class_with_args = RayClassWithInitArgs(cls=GPUAccumulator)
work_group = RayWorkerGroup(resource_pool=resource_pool,ray_cls_with_init=class_with_args)

value = work_group.execute_all_sync("add", x=[1,1])
print(value)

[tensor([1.]), tensor([2.])]


In [13]:
work_group.worker_names,work_group.world_size

(['IOycAUGPUAccumulator_0:0', 'IOycAUGPUAccumulator_0:1'], 2)

In [15]:
from verl.single_controller.base.decorator import Dispatch, Execute, register


In [16]:
@ray.remote
class GPUAccumulatorDecorator(Worker):
    def __init__(self):
        super().__init__()
        self.value = torch.zeros(1, device="cuda") + self.rank


    @register(dispatch_mode=Dispatch.ONE_TO_ALL)
    def add(self, x):
        self.value += x
        print(f"rank {self.rank}, value: {self.value}")
        return self.value.cpu()
    

class_with_args = RayClassWithInitArgs(cls=GPUAccumulatorDecorator)
work_group = RayWorkerGroup(resource_pool=resource_pool,ray_cls_with_init=class_with_args)

In [ ]:
value = work_group.add(x=10)
value

[tensor([10.]), tensor([11.])]

: 